In [1]:
import os

In [2]:
%pwd

'f:\\DATA SCIENCE\\Projects\\Kidney_Disease DL\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\DATA SCIENCE\\Projects\\Kidney_Disease DL'

In [ ]:
import dagshub
dagshub.init(repo_owner='Adroit10', repo_name='Kidney_Disease_DL', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [5]:
!pip install dagshub

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached treelib-1.7.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached pathvalidate-3.2.3-py3-none-any.whl.metadata (12 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached graphql_core-3.2.6-py3-none-any.whl.metadata (11 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
U

In [6]:
!pip install mlflow

  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/29.0 MB ? eta -:--:--
   -----------------------------

In [7]:
import tensorflow as tf
model= tf.keras.models.load_model('artifacts/training/model.h5')


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_size:list
    params_batch_size:int

In [9]:
from src.DiseaseClassifier.constants import *
from src.DiseaseClassifier.utils.common import read_yaml,create_directories,save_json

In [15]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data = "artifacts\data_ingestion\kidney ct-scan images",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )
          return eval_config
    

In [22]:
from urllib.parse import urlparse
import os,certifi
os.environ["REQUESTS_CA_BUNDLE"] = os.environ["SSL_CERT_FILE"] = certifi.where()
import dagshub
import mlflow.keras
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {'loss':self.score[0],'accuracy':self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):

        dagshub.init(repo_owner='Adroit10', repo_name='Kidney_Disease_DL', mlflow=True)
        mlflow.set_registry_uri(mlflow.get_tracking_uri())
        tracking_scheme = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {'loss':self.score[0],'accuracy':self.score[1]}
            )

        if tracking_scheme !='file':
            mlflow.keras.log_model(
                self.model,
                artifact_path='model',
                registered_model_name="VGG16Model"
            )
        else:
            mlflow.keras.log_model(self.model,"model")

In [23]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-05-22 21:43:48,166: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-22 21:43:48,179: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-22 21:43:48,183: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
74/74 [==============================] - 160s 2s/step - loss: 1.3855 - accuracy: 0.8256
[2025-05-22 21:46:29,126: INFO: common: Json file saved at scores.json]
[2025-05-22 21:46:29,599: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Adroit10/Kidney_Disease_DL "HTTP/1.1 200 OK"]
Initialized MLflow to track repo "Adroit10/Kidney_Disease_DL"
[2025-05-22 21:46:29,632: INFO: helpers: Initialized MLflow to track repo "Adroit10/Kidney_Disease_DL"]
Repository Adroit10/Kidney_Disease_DL initialized!
[2025-05-22 21:46:29,633: INFO: helpers: Repository Adroit10/Kidney_Disease_DL initialized!]
🏃 View run resilient-sloth-87 at: https://dagshub.com/Adroit10/Kidney_Disease_DL.mlflow/#/experiments/

2025/05/22 21:46:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpkkepnblj\model\data\model\assets
[2025-05-22 21:46:34,250: INFO: builder_impl: Assets written to: C:\Users\HP\AppData\Local\Temp\tmpkkepnblj\model\data\model\assets]


2025/05/22 21:46:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/05/22 21:47:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
